<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 20s 763ms/step - loss: 0.6688 - accuracy: 0.5888 - val_loss: 0.6563 - val_accuracy: 0.8000
Epoch 2/10
26/26 [==============================] - 19s 734ms/step - loss: 0.6283 - accuracy: 0.6959 - val_loss: 0.6088 - val_accuracy: 0.7561
Epoch 3/10
26/26 [==============================] - 19s 723ms/step - loss: 0.5946 - accuracy: 0.7092 - val_loss: 0.6141 - val_accuracy: 0.5951
Epoch 4/10
26/26 [==============================] - 19s 734ms/step - loss: 0.5495 - accuracy: 0.7324 - val_loss: 0.5058 - val_accuracy: 0.8098
Epoch 5/10
26/26 [==============================] - 19s 731ms/step - loss: 0.4694 - accuracy: 0.8345 - val_loss: 0.5173 - val_accuracy: 0.7463
Epoch 6/10
26/26 [==============================] - 19s 735ms/step - loss: 0.4457 - accuracy: 0.7871 - val_loss: 0.4045 - val_accuracy: 0.8829
Epoch 7/10
26/26 [==============================] - 19s 731ms/step - loss: 0.3628 - accuracy: 0.8650 - val_loss: 0.3202 - val_accuracy: 0.8878

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.71.h5
26/26 - 20s - loss: 0.6744 - accuracy: 0.5876 - val_loss: 0.7089 - val_accuracy: 0.4341
Epoch 2/5

Epoch 00002: saving model to weights.02-0.61.h5
26/26 - 19s - loss: 0.6353 - accuracy: 0.6691 - val_loss: 0.6108 - val_accuracy: 0.8000
Epoch 3/5

Epoch 00003: saving model to weights.03-0.54.h5
26/26 - 19s - loss: 0.5772 - accuracy: 0.7640 - val_loss: 0.5445 - val_accuracy: 0.8341
Epoch 4/5

Epoch 00004: saving model to weights.04-0.52.h5
26/26 - 19s - loss: 0.5181 - accuracy: 0.7737 - val_loss: 0.5232 - val_accuracy: 0.7463
Epoch 5/5

Epoch 00005: saving model to weights.05-0.43.h5
26/26 - 19s - loss: 0.4750 - accuracy: 0.8005 - val_loss: 0.4349 - val_accuracy: 0.7951


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 20s - loss: 0.6695 - accuracy: 0.5912 - val_loss: 0.7307 - val_accuracy: 0.4341


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 20s - loss: 0.6683 - accuracy: 0.5779 - val_loss: 0.6800 - val_accuracy: 0.4780
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 20s - loss: 0.5913 - accuracy: 0.7007 - val_loss: 0.5494 - val_accuracy: 0.7854


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 20s - loss: 0.6730 - accuracy: 0.5961 - val_loss: 0.6738 - val_accuracy: 0.4780
Epoch 2/50
26/26 - 19s - loss: 0.6270 - accuracy: 0.6776 - val_loss: 0.6634 - val_accuracy: 0.4976
Epoch 3/50
26/26 - 19s - loss: 0.6033 - accuracy: 0.6727 - val_loss: 0.6592 - val_accuracy: 0.5073
Epoch 4/50
26/26 - 19s - loss: 0.5417 - accuracy: 0.7518 - val_loss: 0.4828 - val_accuracy: 0.8537
Epoch 5/50
26/26 - 19s - loss: 0.4699 - accuracy: 0.8212 - val_loss: 0.4056 - val_accuracy: 0.9024
Epoch 6/50
26/26 - 19s - loss: 0.4018 - accuracy: 0.8455 - val_loss: 0.4163 - val_accuracy: 0.8098
Epoch 7/50
26/26 - 19s - loss: 0.3299 - accuracy: 0.8978 - val_loss: 0.2920 - val_accuracy: 0.8927
Epoch 8/50
26/26 - 19s - loss: 0.2453 - accuracy: 0.9331 - val_loss: 0.1785 - val_accuracy: 0.9756
Epoch 9/50
26/26 - 19s - loss: 0.2173 - accuracy: 0.9234 - val_loss: 0.1707 - val_accuracy: 0.9659
Epoch 10/50
26/26 - 19s - loss: 0.1490 - accuracy: 0.9635 - val_loss: 0.1185 - val_accuracy: 0.9805
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 20s 753ms/step - loss: 0.6766 - accuracy: 0.5535 - val_loss: 0.6989 - val_accuracy: 0.4390
Epoch 2/5
26/26 [==============================] - 19s 737ms/step - loss: 0.6207 - accuracy: 0.6691 - val_loss: 0.6151 - val_accuracy: 0.6244
Epoch 3/5
26/26 [==============================] - 19s 746ms/step - loss: 0.5551 - accuracy: 0.7555 - val_loss: 0.5108 - val_accuracy: 0.8049
Epoch 4/5
26/26 [==============================] - 19s 727ms/step - loss: 0.4829 - accuracy: 0.8041 - val_loss: 0.5379 - val_accuracy: 0.6976
Epoch 5/5
26/26 [==============================] - 19s 731ms/step - loss: 0.3970 - accuracy: 0.8650 - val_loss: 0.3740 - val_accuracy: 0.8780


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.553528,0.677079,0.439024,0.698866
1,1,0.669100,0.620264,0.624390,0.615074
2,2,0.755474,0.555722,0.804878,0.510836
3,3,0.804136,0.482464,0.697561,0.537929
4,4,0.864964,0.398290,0.878049,0.374006


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 21s 813ms/step - loss: 0.6855 - accuracy: 0.5280 - val_loss: 0.6816 - val_accuracy: 0.5951

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 19s 726ms/step - loss: 0.6682 - accuracy: 0.5827 - val_loss: 0.6787 - val_accuracy: 0.5220

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 19s 723ms/step - loss: 0.6608 - accuracy: 0.5852 - val_loss: 0.6702 - val_accuracy: 0.6000

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 19s 731ms/step - loss: 0.6559 - accuracy: 0.6375 - val_loss: 0.6711 - val_accuracy: 0.5561

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 19s 735ms/step - loss: 0.6

In [19]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6006 (pid 13103), started 0:17:02 ago. (Use '!kill 13103' to kill it.)

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [20]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 19s 748ms/step - loss: 0.6743 - accuracy: 0.5754 - val_loss: 0.6764 - val_accuracy: 0.5024
Epoch 2/50
25/26 [===========================>..] - ETA: 0s - loss: 0.6539 - accuracy: 0.6288
Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
26/26 [==============================] - 19s 723ms/step - loss: 0.6528 - accuracy: 0.6338 - val_loss: 0.6934 - val_accuracy: 0.4488
Epoch 3/50
26/26 [==============================] - 19s 716ms/step - loss: 0.6373 - accuracy: 0.6472 - val_loss: 0.6520 - val_accuracy: 0.5902
Epoch 4/50
26/26 [==============================] - 19s 731ms/step - loss: 0.6272 - accuracy: 0.7263 - val_loss: 0.6488 - val_accuracy: 0.5902
Epoch 5/50
26/26 [==============================] - 19s 743ms/step - loss: 0.6196 - accuracy: 0.7117 - val_loss: 0.6329 - val_accuracy: 0.6537
Epoch 6/50
26/26 [==============================] - 19s 734ms/step - loss: 0.6115 - accuracy: 0.7287 - val_loss: 0.6225 - 

In [21]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6006 (pid 13103), started 0:32:46 ago. (Use '!kill 13103' to kill it.)